In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DATABASE_URI
from models import *
from sqlalchemy import Column, Table
from scrapping_sources.Macrotrend import Macrotrend
from scrapping_sources.Nasdaq import Nasdaq
from models import Base, Lambda_logs, Statements_table_log
import datetime
import pandas as pd

  lastYearRptDt lastYearEPS             time symbol                      name  \
0    10/09/2020          $0  time-pre-market   BYRN  Byrna Technologies, Inc.   

      marketCap fiscalQuarterEnding epsForecast noOfEsts  
0  $500,112,641            Aug/2021     ($0.04)        2  


In [4]:

USERNAME='postgres'
HOST='database-webscrap.ccaaerr6cq44.eu-west-3.rds.amazonaws.com'
DATABASE_NAME='webscrapping'
PASSWORD='postgres5678'
PORT=5432
DATABASE_URI = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"\

session = sessionmaker(bind=create_engine(DATABASE_URI))
s = session()
# s.close_all()

/tmp/ipykernel_6263/3373429083.py:10: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [21]:
s.close_all()

/tmp/ipykernel_195431/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [17]:
def check_function_status(function):
    "returns the status of a lambda_function"

    try:
        r = s.query(Lambda_logs.status).filter(
            Lambda_logs.lambda_function    == function,
            Lambda_logs.date        == '2021-10-02'
        ).all()
        return r[0][0]
    except:
        return None
check_function_status("ws_update_statements")

'completed'

In [18]:
def no_table_log(ticker, stmnt, t_format, period):

    r = s.query(Statements_table_log.__table__).filter(
        Statements_table_log.ticker == ticker,
        Statements_table_log.statement == stmnt,
        Statements_table_log.time_format == t_format,
        Statements_table_log.status == period
    ).all()

    return len(r) == 0

no_table_log('AAPL', "balance-sheet", "annual", Macrotrend().latest_ending_period_available('AAPL'))

AAPL <Response [200]>


True

In [62]:
def update_statements_log_entry(ticker, stmnt, t_format, period, status):


    s.query(Statements_table_log.__table__).filter(
        Statements_table_log.ticker == ticker,
        Statements_table_log.statement == stmnt,
        Statements_table_log.time_format == t_format,
        Statements_table_log.period == period). \
        update({"status": f"{status}"})
    s.commit()


In [63]:
update_statements_log_entry('MKC', 'income-statement', 'annual', '2021-08', 'test')

AttributeError: type object 'Statements_table_log' has no attribute 'period'

In [1]:
s.close_all()

NameError: name 's' is not defined

In [33]:
s.query(Earnings_release.__table__).\
       filter(Earnings_release.id == 38).\
       update({"last_period_M": '2021-03'})
s.commit()

In [8]:
statements = [
'income_statement',
'balance_sheet',
'cash_flow_statement',
'financial_ratios'
]
time_format = [
    'annual',
    'quarterly'
]
def has_f_data_in_db(ticker):

    data_boolean_list = []
    for stmnt in statements:
        for t_format in time_format:
            q = s.query(Base.metadata.tables[f'{stmnt}_{t_format}']).where(
                Base.metadata.tables[f'{stmnt}_{t_format}'].columns['ticker'] ==str(ticker)).all()
            print(q)
            if len(q) == 0:
                data_boolean_list.append(False)
            else:
                data_boolean_list.append(True)
    return any(data_boolean_list)

def fetch_all_statement(ticker, stmnt, time_format):

    r = s.query(Base.metadata.tables[f'{stmnt}_{time_format}']).where(\
            Base.metadata.tables[f'{stmnt}_{time_format}'].columns['ticker'] ==str(ticker)).all()
    return r

pd.DataFrame(fetch_all_statement('AAPL', 'balance_sheet', 'annual'))

            

,0,1,2,3,4,5,6
0,3917,2020-09-30,BS-A,AAPL,17,Receivables,37445.0
1,3918,2020-09-30,BS-A,AAPL,17,Inventory,4061.0
2,3919,2020-09-30,BS-A,AAPL,17,Pre-Paid Expenses,None
3,3920,2020-09-30,BS-A,AAPL,17,Other Current Assets,11264.0
4,3921,2020-09-30,BS-A,AAPL,17,Total Current Assets,143713.0
...,...,...,...,...,...,...,...
347,4264,2005-09-30,BS-A,AAPL,17,Retained Earnings (Accumulated Deficit),3925.0
348,4265,2005-09-30,BS-A,AAPL,17,Comprehensive Income,None
349,4266,2005-09-30,BS-A,AAPL,17,Other Share Holders Equity,None
350,4267,2005-09-30,BS-A,AAPL,17,Share Holder Equity,7428.0


In [88]:

M = Macrotrend()

def has_data_trend(ticker):
    _ = []

    for stmnt in statements:
        for t_format in time_format:
            r = M.arrange_data(ticker, stmnt.replace('_', '-'), t_format)
            if r is not None:
                _.append(True)
    print(_)
    return any(_)


In [104]:
N = Nasdaq()

def convert_date_N(date):
    return datetime.datetime.strptime(str(date), '%b/%Y').strftime("%Y-%m")
# def last_period_nasdaq(ticker):
data = N.earnings_release(datetime.date.today())
data['fiscalQuarterEnding'].map(convert_date_N)
# lambda x: datetime.datetime.strptime(str(x), '%b/%Y').strftime("%Y-%m")

# datetime.datetime.strptime('Jun/2021', '%b/%Y').strftime("%Y-%m")

0     2021-06
1     2021-06
2     2021-06
3     2021-06
4     2021-06
5     2021-06
6     2021-06
7     2021-06
8     2021-06
9     2021-06
10    2021-06
11    2021-06
12    2021-06
13    2021-06
14    2021-06
15    2021-06
16    2021-06
17    2021-06
18    2021-06
19    2021-06
20    2021-06
Name: fiscalQuarterEnding, dtype: object

In [23]:
querry = s.query(Security.ticker, Security.id).all()
security_id_mapper = {querry[i][0]: querry[i][1] for i, v in zip(range(len(querry)), range(len(querry)))}

In [122]:
r = s.query(Base.metadata.tables['income_statement_quarterly'].columns['date']).where(
    Base.metadata.tables['income_statement_quarterly'].columns['ticker'] == str('AAPL')
).all()
r.sort(reverse=True)
r = str(r[0][0])

def convert_to_ending_period_format( date):
    """Returns """
    result = datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime("%Y-%m")
    return result

convert_to_ending_period_format(r)

'2021-06'

In [123]:
M.latest_ending_period_available('AAPL', 'balance-sheet', 'annual')

AAPL <Response [200]>


'2020-09'

In [25]:
start =time.perf_counter()
finviz_table = None

while finviz_table is None and (time.perf_counter() - start) < 20:
    print((time.perf_counter() - start))
    if (time.perf_counter() - start) >5:
        finviz_table = 1
print(finviz_table)

NameError: name 'time' is not defined

In [ ]:
engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)
s = Session()

In [ ]:
q = s.query(Security.ticker).all()

In [ ]:
[i for i in range(0,75,10)]
# COMPARE WITH MYSQL OUTPUT. MAKE SURE THEY ARE THE SAME

[0, 10, 20, 30, 40, 50, 60, 70]

In [ ]:
my_list = [i for i in range(0,75)]

In [ ]:
[my_list[i:i + 10] for i in range(0, len(my_list), 10)]

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74]]

In [ ]:
def chunks(lst, n):
    chunk_list = []
    for i in range(0, len(lst), n):
        chunk_list.append(lst[i:i + n])
    return chunk_list

In [ ]:
chunks(my_list, 10)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74]]

In [ ]:
s.close_all()

/tmp/ipykernel_534434/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [ ]:
r = s.query(Security.ticker).all()

In [ ]:
tickers =[i[0] for i in r][:100]

In [ ]:
scrapper = Macrotrend()
scrapper.generate_statement_table_multi_threading(tickers, "balance-sheet", "annual")

AAAU <Response [404]>
No data available for AAAU, balance-sheet, annual
AADR <Response [404]>
No data available for AADR, balance-sheet, annual
AADI <Response [200]>
AAIC <Response [200]>
AAL <Response [200]>
AA <Response [200]>
AACG <Response [200]>
AACIU <Response [404]>
No data available for AACIU, balance-sheet, annual
AAC <Response [200]>
A <Response [200]>
AAMC <Response [200]>
AAME <Response [200]>
AAN <Response [200]>
AAPL <Response [200]>
AAON <Response [200]>
AAP <Response [200]>
AAOI <Response [200]>
AAT <Response [200]>
AAQC <Response [200]>
No data available for AAQC, balance-sheet, annual
AATC <Response [200]>
AAWW <Response [200]>
AB <Response [200]>
ABB <Response [200]>
AAU <Response [200]>
AAXJ <Response [404]>
No data available for AAXJ, balance-sheet, annual
ABBV <Response [200]>
ABCB <Response [200]>
ABCL <Response [200]>
ABC <Response [200]>
ABCM <Response [200]>
ABEO <Response [200]>
ABEQ <Response [404]>
No data available for ABEQ, balance-sheet, annual
ABEV <Res

,date,statement,ticker,security_id,line_item,amount
0,2020-12-31,BS-A,AADI,None,Receivables,NaN
1,2020-12-31,BS-A,AADI,None,Inventory,NaN
2,2020-12-31,BS-A,AADI,None,Pre-Paid Expenses,0.5102
3,2020-12-31,BS-A,AADI,None,Other Current Assets,1.6039
4,2020-12-31,BS-A,AADI,None,Total Current Assets,44.7190
...,...,...,...,...,...,...
39,2019-12-31,BS-A,ACVA,None,Retained Earnings (Accumulated Deficit),-125.9580
40,2019-12-31,BS-A,ACVA,None,Comprehensive Income,-0.0010
41,2019-12-31,BS-A,ACVA,None,Other Share Holders Equity,NaN
42,2019-12-31,BS-A,ACVA,None,Share Holder Equity,205.3260


AttributeError: type object 'Statements_table_log' has no attribute 'period'

In [ ]:
table = []

def create_table(ticker, statement, time_format):

    df = scrapper.arrange_data(ticker, statement, time_format)
    if isinstance(df, pd.DataFrame):
        table.append(df)


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

    executor.map(create_table, tickers, repeat('balance-sheet'), repeat("annual"))

# table
table_concat = pd.concat(table)
table_concat['amount'] =  pd.to_numeric(table_concat['amount'])


AAIC <Response [200]>
AAC <Response [200]>
AAAU <Response [404]>
No data available for AAAU, balance-sheet, annual
AADI <Response [200]>
AAL <Response [200]>
AADR <Response [404]>
No data available for AADR, balance-sheet, annual
A <Response [200]>
AA <Response [200]>
AACIU <Response [404]>
No data available for AACIU, balance-sheet, annual
AACG <Response [200]>
AAMC <Response [200]>
AAME <Response [200]>
AAN <Response [200]>
AAP <Response [200]>
AAOI <Response [200]>
AAON <Response [200]>
AAQCAAXJ <Response [404]>
No data available for AAXJ, balance-sheet, annual
 AAT <Response [200]>
AAU <Response [200]>
AAWW <Response [200]>
AATC <Response [200]>
AAPL <Response [200]><Response [200]>
No data available for AAQC, balance-sheet, annual

ABC <Response [200]>
ABB <Response [200]>
AB <Response [200]>
ABBV <Response [200]>
ABCBABEQ <Response [200]>
 <Response [404]>
No data available for ABEQ, balance-sheet, annual
ABCL <Response [200]>
ABEO <Response [200]>
ABCM <Response [200]>
ABEV <Res

In [ ]:
table_concat

,date,statement,ticker,security_id,line_item,amount
0,2020-12-31,BS-A,AAMC,None,Receivables,3.414
1,2020-12-31,BS-A,AAMC,None,Inventory,NaN
2,2020-12-31,BS-A,AAMC,None,Pre-Paid Expenses,3.328
3,2020-12-31,BS-A,AAMC,None,Other Current Assets,48.249
4,2020-12-31,BS-A,AAMC,None,Total Current Assets,96.614
...,...,...,...,...,...,...
39,2019-12-31,BS-A,ACVA,None,Retained Earnings (Accumulated Deficit),-125.958
40,2019-12-31,BS-A,ACVA,None,Comprehensive Income,-0.001
41,2019-12-31,BS-A,ACVA,None,Other Share Holders Equity,NaN
42,2019-12-31,BS-A,ACVA,None,Share Holder Equity,205.326


In [ ]:
scrapper.get_statement('AAPL', 'balance-sheet', 'annual')

AAPL <Response [200]>


,field_name,2020-09-30,2019-09-30,2018-09-30,2017-09-30,2016-09-30,2015-09-30,2014-09-30,2013-09-30,2012-09-30,2011-09-30,2010-09-30,2009-09-30,2008-09-30,2007-09-30,2006-09-30,2005-09-30
0,Cash On Hand,90943.00000,100557.00000,66301.00000,74181.00000,67155.00000,41601.00000,25077.00000,40546.00000,29129.00000,25952.00000,25620.00000,23464.00000,22111.00000,15386.00000,10110.00000,8261.00000
1,Receivables,37445.00000,45804.00000,48995.00000,35673.00000,29299.00000,30343.00000,27219.00000,20641.00000,18692.00000,11717.00000,9924.00000,5057.00000,2422.00000,1637.00000,1252.00000,895.00000
2,Inventory,4061.00000,4106.00000,3956.00000,4855.00000,2132.00000,2349.00000,2111.00000,1764.00000,791.00000,776.00000,1051.00000,455.00000,509.00000,346.00000,270.00000,165.00000
3,Pre-Paid Expenses,,,,,,,,,,,,,,,,
4,Other Current Assets,11264.00000,12352.00000,12087.00000,13936.00000,8283.00000,15085.00000,9806.00000,6882.00000,6458.00000,4529.00000,3447.00000,1444.00000,3920.00000,3805.00000,2270.00000,648.00000
5,Total Current Assets,143713.00000,162819.00000,131339.00000,128645.00000,106869.00000,89378.00000,68531.00000,73286.00000,57653.00000,44988.00000,41678.00000,31555.00000,30006.00000,21956.00000,14509.00000,10300.00000
6,"Property, Plant, And Equipment",36766.00000,37378.00000,41304.00000,33783.00000,27010.00000,22471.00000,20624.00000,16597.00000,15452.00000,7777.00000,4768.00000,2954.00000,2455.00000,1832.00000,1281.00000,817.00000
7,Long-Term Investments,100887.00000,105341.00000,170799.00000,194714.00000,170430.00000,164065.00000,130162.00000,106215.00000,92122.00000,55618.00000,25391.00000,10528.00000,2379.00000,,,
8,Goodwill And Intangible Assets,,,,,8620.00000,9009.00000,8758.00000,5756.00000,5359.00000,4432.00000,1083.00000,453.00000,492.00000,337.00000,177.00000,96.00000
9,Other Long-Term Assets,42522.00000,32978.00000,22283.00000,18177.00000,8757.00000,5422.00000,3764.00000,5146.00000,5478.00000,3556.00000,2263.00000,2011.00000,839.00000,1222.00000,1238.00000,303.00000


In [ ]:
s.query(Security.ticker).filter_by(ticker="jlkkj").first()[0]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from models import Base

In [ ]:
s.query(Base.metadata.tables['income_statement_annual'].columns.ticker).filter_by(ticker='A').all()

[('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 

In [ ]:
s.close_all()

/tmp/ipykernel_68765/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [ ]:
A = ('a','b','c')
B = ('a','b','c')
C = ('d','e','f')
D = ('g','h','i')

In [ ]:
my_list = [A,B,C]

In [ ]:
set_1 = set(my_list)
set_2 = set([C,D])

In [ ]:
len(set_1-set_2)

1